## We need the optimal strategy to load/save models

In [8]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.expandvars("/home/$USER/repos/fande/") )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import torch
from fande.ase import FandeCalc

predictor_loaded = torch.load("/data1/simulations/ML_models/predictor.pt")
fande_calc = FandeCalc(predictor_loaded)

atoms = predictor_loaded.fdm.traj_train[0].copy()
atoms.calc = fande_calc

print( atoms.get_potential_energy(), atoms.get_forces() )


/home/dlbox2/repos/fande/fande/predict/predictors_ase.py:571: UserWarning:

Calculating invariants...



Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


100%|██████████| 1/1 [00:00<00:00,  1.54it/s]

0.0 [[ 3.23001528 -1.33135843 -1.74239302]
 [-0.546278   -1.22240639  0.35187721]
 [-0.1907053  -1.89948273 -0.81701851]
 ...
 [ 0.48821259 -1.30752563  0.28121185]
 [ 0.23548126  0.74869919  1.14985275]
 [ 0.92536926 -1.72566223 -0.75263214]]



/home/dlbox2/repos/fande/fande/predict/predictors_ase.py:595: UserWarning:

Predicting...



In [20]:
from ase import io

traj_295 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index=":")
traj_355 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")

traj_295_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
traj_355_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
traj_295_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
traj_355_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")

In [13]:
from fande.utils.test import make_test_plots

In [21]:
save_dir_295 = "/data1/simulations/ML_models/test_plots/295"
save_dir_355 = "/data1/simulations/ML_models/test_plots/355"
save_dir_295_2000 = "/data1/simulations/ML_models/test_plots/295_2000"
save_dir_355_2000 = "/data1/simulations/ML_models/test_plots/295_2000"
save_dir_295_2000_forced = "/data1/simulations/ML_models/test_plots/295_2000_forced"
save_dir_355_2000_forced = "/data1/simulations/ML_models/test_plots/355_2000_forced"

In [ ]:
trajectories = [traj_295, traj_355, traj_295_2000K, traj_355_2000K, traj_295_2000K_forced, traj_355_2000K_forced]
save_dirs = [save_dir_295, save_dir_355, save_dir_295_2000, save_dir_355_2000, save_dir_295_2000_forced, save_dir_355_2000_forced]

In [ ]:
for traj, save_dir in zip(trajectories, save_dirs):
    make_test_plots(traj, fande_calc, save_dir=save_dir)

In [25]:
%%capture c
make_test_plots(fande_calc, traj_295[4000:4100], save_dir=save_dir[0])